In [ ]:
#!/usr/bin/python

# Description:   

In [1]:
# --- Setup --- #
import os
import re
from bs4 import BeautifulSoup
import json
import parser as ps

In [24]:
def item_one_extract_1(txt):
    """
    Extract Item One from raw txt files
    
    Input:
        cleaned text
    Output:
        item one
        -1 - has multiple match
        -2 - stop sign broken (Add a fixer module)
    """
    reg1 = re.compile(r'^(\s*)i(\s*)t(\s*)e(\s*)m(\s*|\n*)1[\.|\s|:](.|\n)*?(^(\s*)i(\s*)t(\s*)e(\s*)m(\s*|\n*)1(\.?)(\s*|\n*)a[\.|\s|:]).*?\n', 
                      flags = re.I|re.M)
    reg2 = re.compile(r'^(\s*)i(\s*)t(\s*)e(\s*)m(\s*|\n*)1[\.|\s|:](.|\n)*?(^(\s*)i(\s*)t(\s*)e(\s*)m(\s*|\n*)2[\.|\s|:]).*?\n', 
                  flags = re.I|re.M)
    reg3 = re.compile(r'^(\s*)i(\s*)t(\s*)e(\s*)m(\s*|\n*)s(\s*|\n*)1(\.?)(\s*|\n*)a(\s*|\n*)n(\s*|\n*)d(\s*|\n*)2[\.|\s|:](.|\n)*?(^(\s*)i(\s*)t(\s*)e(\s*)m(\s*|\n*)1(\s*|\n*|\.)a[\.|\s|:]).*?\n', 
                      flags = re.I|re.M)
    
    res = []
    itr1 = reg1.finditer(txt)
    
    for i in itr1:
        if len(i.group(0)) > 500:
            res.append(i.group(0))
        
    if len(res) == 1:
        endHook = re.search(r'^(\s*)i(\s*)t(\s*)e(\s*)m(\s*|\n*)1(\s*|\n*)a[\.|\s|:].*?\n', res[0], 
                            flags = re.I|re.M)
        endStr = endHook.group(0).strip()
        if len(endStr) > 100:
            if re.search(r'risk(\s*)factor(s?)\s{2,}', endStr, flags = re.I):
                return(res[0].replace(endStr, '').strip())
            else:
                return(-2)
        else:
            item_one = res[0].replace(endStr, '')
            return(item_one.strip())
    elif len(res) == 0:
        # Now try the second parsing algo
        itr2 = reg3.finditer(txt)
        for i in itr2:
            if len(i.group(0)) > 500:
                res.append(i.group(0))
        if len(res) == 1:
            endHook = re.search(r'^(\s*)i(\s*)t(\s*)e(\s*)m(\s*|\n*)1(\s*|\n*)a[\.|\s|:].*?\n', res[0], 
                            flags = re.I|re.M)
            endStr = endHook.group(0).strip()
            if len(endStr) > 100:
                if re.search(r'risk(\s*)factor(s?)\s{2,}', endStr, flags = re.I):
                    return(res[0].replace(endStr, '').strip())
                else:
                    return(-2)
            else:
                item_one = res[0].replace(endStr, '')
                return(item_one.strip())
        else:
            return(res)
    elif len(res) == 2:
        endHook = re.search(r'^(\s*)i(\s*)t(\s*)e(\s*)m(\s*|\n*)1(\s*|\n*)a[\.|\s|:].*?\n', res[1], 
                            flags = re.I|re.M)
        endStr = endHook.group(0).strip()
        if len(endStr) > 100:
            if re.search(r'risk(\s*)factor(s?)\s{2,}', endStr, flags = re.I):
                return(res[1].replace(endStr, '').strip())
            else:
                return(-2)
        else:
            item_one = res[1].replace(endStr, '')
            return(item_one.strip())
    else:
        return(-1)

In [3]:
def validate(txt):
    """
    Validate that item one is correctly extracted
    
    Input:
        item one
    Output:
        result variable
             1 - pass validation test
             0 - the pasrsing is failed
            -1 - item one is none
            -2 - item one is too short
            
            
    """
    if txt is None:
        return(-1)
    
    if len(txt) < 500:
        return(-2)
    
    # Normalize string
    tester = txt[len(txt)-1000:len(txt)].lower()
    tester = re.sub(r'table of content(s?)', '', tester, flags = re.IGNORECASE)
    tester = re.sub(r'\s\d{1,}\s', '', tester)
    tester = re.sub(r'[^A-Za-z0-9\.]', '', tester)
    tester = re.sub(r'\d*$', '', tester)
          
    # Check if the end is a period
    checker = re.findall(r'\.$', tester)
    
    if len(checker) > 0:
        return(1)
    else:
        stopper = re.search(r'\.[^\.]*?$', tester).group(0) 
        stopper = re.sub(r'index', '', stopper, flags = re.I)
        stopper = re.sub(r'financialstatement(s?)', '', stopper, flags = re.I)
        stopper = re.sub(r'\d*', '', stopper)
        if len(stopper) < 30:
            return(1)
        else:
            return(stopper)

In [18]:
# ==== Main ==== #
# A typical workflow for parsing
os.chdir('/yuan/10k/New10K/')  # move to local directory

fff = 'raw/2015/' + '1005409_0001005409-15-000015.txt'
txt = ps.clean_text(ps.load_file(fff))

In [25]:
res = item_one_extract_1(txt)
res

'Item 1. Business\n\nGeneral\n\nThe Holding Company is a Delaware corporation and parent company of the Bank, a New York State chartered savings bank. The Bank\'s principal business is gathering retail deposits, and lending them primarily in multifamily residential, commercial real estate and mixed use loans, as well as investing in mortgage-backed securities ("MBS"), obligations of the U.S. Government and Government Sponsored Entities ("GSEs"), and corporate debt and equity securities. The Bank\'s revenues are derived principally from interest on its loan and securities portfolios, and other investments. The Bank\'s primary sources of funds are, in general, deposits; loan amortization, prepayments and maturities; MBS amortization, prepayments and maturities; investment securities maturities and sales; and advances from the Federal Home Loan Bank of New York ("FHLBNY").\n\nThe primary business of the Holding Company is the ownership of its wholly-owned subsidiary, the Bank. The Holding

In [13]:
validate(res)

'.eachofthesedocumentsisalsoavailableinprinttoanystockholderwhorequestsacopybyaddressingarequesttolennarcorporationattentionofficeofthegeneralcounselnorthwestthavenuemiamiflorida'